In [ ]:
#| default_exp hier

# hier

> Code for dealing with multiindexed columns

In [ ]:
#| export
import pathlib
import itertools

import pandas as pd
import numpy as np
from lxml import etree
import yaml

# import sproc.xml
import sproc.structure

In order to avoid circular dependencies in the resulting *Python* modules, and since this is only for testing

In [ ]:
import sproc.xml

Directory where the data (*XML* files) are stored

In [ ]:
directory = pathlib.Path.cwd().parent / 'samples'
assert directory.exists()
directory

PosixPath('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples')

A (sample) file in that directory

In [ ]:
xml_file = directory / 'PlataformasAgregadasSinMenores_20220104_030016_1.atom'
assert xml_file.exists()
xml_file

PosixPath('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/PlataformasAgregadasSinMenores_20220104_030016_1.atom')

*Root* element of the *XML* tree

In [ ]:
root = etree.parse(xml_file).getroot()

Entries are extracted

In [ ]:
entries = sproc.xml.get_entries(root)
assert len(entries) == 117
entries[:4]

[<Element {http://www.w3.org/2005/Atom}entry>,
 <Element {http://www.w3.org/2005/Atom}entry>,
 <Element {http://www.w3.org/2005/Atom}entry>,
 <Element {http://www.w3.org/2005/Atom}entry>]

The first one is picked out

In [ ]:
element = entries[0]
sproc.xml.split_namespace_tag(element.tag)

('http://www.w3.org/2005/Atom', 'entry')

It is turned into a `pd.Series`

In [ ]:
element_series = sproc.xml.entry_to_series(element)
element_series[:8]

id                                                                           https://contrataciondelestado.es/sindicacion/P...
summary                                                                      Id licitación: C. 2-2021; Órgano de Contrataci...
title                                                                        L'objecte del contracte és la renovació de tot...
updated                                                                                          2022-01-03T01:11:41.826+01:00
ContractFolderStatus - ContractFolderID                                                                              C. 2-2021
ContractFolderStatus - ContractFolderStatusCode                                                                            ADJ
ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID           https://contractaciopublica.gencat.cat/ecofin_...
ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name                             Ajuntamen

## Series

We can easily iterate the series

In [ ]:
next(iter(element_series.items()))

('id',
 'https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/8799346')

In [ ]:
index_paths = []
values = []
for i, v in element_series.items():
    index_paths.append(tuple(i.split(sproc.structure.nested_tags_separator)))
    values.append(v)

In [ ]:
index_paths[:6]

[('id',),
 ('summary',),
 ('title',),
 ('updated',),
 ('ContractFolderStatus', 'ContractFolderID'),
 ('ContractFolderStatus', 'ContractFolderStatusCode')]

In [ ]:
values[:6]

['https://contrataciondelestado.es/sindicacion/PlataformasAgregadasSinMenores/8799346',
 'Id licitación: C. 2-2021; Órgano de Contratación: Ajuntament de Sant Ramon; Importe: 135553.26; Estado: ADJUDICADA',
 "L'objecte del contracte és la renovació de totes les llumeneres que formen la il·luminació existent de tots els carrers i vials del casc urbà de la localitat de Sant Ramon i dels nuclis agregats de La Manresana, Portell, Viver i Gospí",
 '2022-01-03T01:11:41.826+01:00',
 'C. 2-2021',
 'ADJ']

In [ ]:
multiindex = pd.MultiIndex.from_tuples(index_paths)
multiindex[:6]

MultiIndex([(                  'id',                        nan, nan, ...),
            (             'summary',                        nan, nan, ...),
            (               'title',                        nan, nan, ...),
            (             'updated',                        nan, nan, ...),
            ('ContractFolderStatus',         'ContractFolderID', nan, ...),
            ('ContractFolderStatus', 'ContractFolderStatusCode', nan, ...)],
           )

In [ ]:
multiindexed_series = pd.Series(values, index=multiindex)
multiindexed_series[:6]

id                    NaN                       NaN  NaN  NaN    https://contrataciondelestado.es/sindicacion/P...
summary               NaN                       NaN  NaN  NaN    Id licitación: C. 2-2021; Órgano de Contrataci...
title                 NaN                       NaN  NaN  NaN    L'objecte del contracte és la renovació de tot...
updated               NaN                       NaN  NaN  NaN                        2022-01-03T01:11:41.826+01:00
ContractFolderStatus  ContractFolderID          NaN  NaN  NaN                                            C. 2-2021
                      ContractFolderStatusCode  NaN  NaN  NaN                                                  ADJ
dtype: object

A function that turns the index of a `pd.Series` into a hierarchical (`pd.MultiIndex`) one.

In [ ]:
#| export
def flat_series_to_multiindexed_series(
    s: pd.Series # Flat series
) -> pd.Series: # Multi-indexed series
    "Returns a multi-indexed version of the input"
    
    index_paths = []
    values = []
    
    for i, v in s.items():
        index_paths.append(tuple(i.split(sproc.structure.nested_tags_separator)))
        values.append(v)
        
    return pd.Series(values, index=pd.MultiIndex.from_tuples(index_paths))

In [ ]:
flat_series_to_multiindexed_series(element_series)[:6]

id                    NaN                       NaN  NaN  NaN    https://contrataciondelestado.es/sindicacion/P...
summary               NaN                       NaN  NaN  NaN    Id licitación: C. 2-2021; Órgano de Contrataci...
title                 NaN                       NaN  NaN  NaN    L'objecte del contracte és la renovació de tot...
updated               NaN                       NaN  NaN  NaN                        2022-01-03T01:11:41.826+01:00
ContractFolderStatus  ContractFolderID          NaN  NaN  NaN                                            C. 2-2021
                      ContractFolderStatusCode  NaN  NaN  NaN                                                  ADJ
dtype: object

## DataFrame

We can concatenate together the `pd.Series` for the different *entries* into a `pd.DataFrame`

In [ ]:
pd.concat([flat_series_to_multiindexed_series(sproc.xml.entry_to_series(e)) for e in entries[:4]], axis=1).T

id  \
                                                 NaN   
                                                 NaN   
                                                 NaN   
                                                 NaN   
0  https://contrataciondelestado.es/sindicacion/P...   
1  https://contrataciondelestado.es/sindicacion/P...   
2  https://contrataciondelestado.es/sindicacion/P...   
3  https://contrataciondelestado.es/sindicacion/P...   

                                             summary  \
                                                 NaN   
                                                 NaN   
                                                 NaN   
                                                 NaN   
0  Id licitación: C. 2-2021; Órgano de Contrataci...   
1  Id licitación: 8128_3/2021; Órgano de Contrata...   
2  Id licitación: 1000_0005-CP01-2021-000063; Órg...   
3  Id licitación: 1379/2020 4738; Órgano de Contr...   

                                               title  \
                                                 NaN   
                                                 NaN   
                                                 NaN   
                                                 NaN   
0  L'objecte del contracte és la renovació de tot...   
1  Obras de restauración hidromorfológica del río...   
2  Contrato del servicio de realización de labore...   
3  Obres de renovació de l'enllumenat públic a la...   

                         updated        ContractFolderStatus  \
                             NaN            ContractFolderID   
                             NaN                         NaN   
                             NaN                         NaN   
                             NaN                         NaN   
0  2022-01-03T01:11:41.826+01:00                   C. 2-2021   
1  2022-01-03T01:00:11.194+01:00                 8128_3/2021   
2  2022-01-03T01:00:10.399+01:00  1000_0005-CP01-2021-000063   
3  2022-01-03T00:11:40.740+01:00              1379/2020 4738   

                                                                               \
  ContractFolderStatusCode                            LocatedContractingParty   
                       NaN                                  BuyerProfileURIID   
                       NaN                                                NaN   
                       NaN                                                NaN   
0                      ADJ  https://contractaciopublica.gencat.cat/ecofin_...   
1                      PUB                                                NaN   
2                       EV                                                NaN   
3                       EV  https://contractaciopublica.gencat.cat/ecofin_...   

                                                      \
                                                       
                                               Party   
                                           PartyName   
                                                Name   
0                           Ajuntament de Sant Ramon   
1                             Pleno del Ayuntamiento   
2  El Director General de Comunicación y Relacion...   
3                         Ajuntament de Canet de Mar   

                                                      \
                                                       
                                  ParentLocatedParty   
                                           PartyName   
                                                Name   
0                   Entitats municipals de Catalunya   
1                            AYUNTAMIENTO DE MONREAL   
2  Departamento de Presidencia, Igualdad, Función...   
3                   Entitats municipals de Catalunya   

                                                      ...  \
                                  ProcurementProject  ...   
                                                Name  ...   
                                   

However, there is a problem if the `pd.Series`'s to be concatenated have different number of levels

In [ ]:
flat_series_to_multiindexed_series(sproc.xml.entry_to_series(entries[15])).index.nlevels

5

In [ ]:
flat_series_to_multiindexed_series(sproc.xml.entry_to_series(entries[16])).index.nlevels

6

In [ ]:
try:
    pd.concat([flat_series_to_multiindexed_series(sproc.xml.entry_to_series(e)) for e in entries[15:17]], axis=1).T
except ValueError:
    print('Oooooooooops')

Oooooooooops


The function below just converts the `pd.DataFrame` *column-wise*. Columns with different levels are handled as suggested in [this post](https://stackoverflow.com/a/68940748).

In [ ]:
#| export
def flat_df_to_multiindexed_df(
    input_df: pd.DataFrame # Input dataframe
) -> pd.DataFrame: # A column-hierarchical version of the input dataframe
    "Reads and parses an XML file into a `DataFrame`"
    
    # every field becomes a `tuple`
    fields = [tuple(c.split(sproc.structure.nested_tags_separator)) for c in input_df.columns]
    
    # the number of levels in the multindex for the columns
    n_levels = len(max(fields, key=len))
    
    # every tuple is padded with empty string until it has `n_levels`
    fields = [e + ('',)*(n_levels-len(e)) for e in fields]

    index_hierarchical = pd.MultiIndex.from_tuples(fields)

    # an empty `pd.DataFrame`
    res = pd.DataFrame(None, columns=index_hierarchical)

    # every column in the *output* `pd.DataFrame`...
    for c in res.columns:

        # ...is filled in looking up the data in the input `pd.DataFrame` by means of the appropriate "merged" column name
        res[c] = input_df[sproc.structure.assemble_name(c)]
    
    return res

We first build the `pd.DataFrame`

In [ ]:
df = sproc.xml.to_df(xml_file)
df.head(6)

,id,summary,title,updated,ContractFolderStatus - ContractFolderID,ContractFolderStatus - ContractFolderStatusCode,ContractFolderStatus - LocatedContractingParty - BuyerProfileURIID,ContractFolderStatus - LocatedContractingParty - Party - PartyName - Name,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - ProcurementProject - Name,...,ContractFolderStatus - LegalDocumentReference - Attachment - ExternalReference - URI,ContractFolderStatus - TechnicalDocumentReference - ID,ContractFolderStatus - TechnicalDocumentReference - Attachment - ExternalReference - URI,ContractFolderStatus - ProcurementProject - PlannedPeriod - StartDate,ContractFolderStatus - ProcurementProject - PlannedPeriod - EndDate,ContractFolderStatus - LocatedContractingParty - Party - PartyIdentification - ID,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndDate,ContractFolderStatus - TenderingProcess - ParticipationRequestReceptionPeriod - EndTime,ContractFolderStatus - TenderResult - AwardedTenderedProject - ProcurementProjectLotID
0,https://contrataciondelestado.es/sindicacion/P...,Id licitación: C. 2-2021; Órgano de Contrataci...,L'objecte del contracte és la renovació de tot...,2022-01-03T01:11:41.826+01:00,C. 2-2021,ADJ,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Sant Ramon,Entitats municipals de Catalunya,L'objecte del contracte és la renovació de tot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8128_3/2021; Órgano de Contrata...,Obras de restauración hidromorfológica del río...,2022-01-03T01:00:11.194+01:00,8128_3/2021,PUB,NaN,Pleno del Ayuntamiento,AYUNTAMIENTO DE MONREAL,Obras de restauración hidromorfológica del río...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1000_0005-CP01-2021-000063; Órg...,Contrato del servicio de realización de labore...,2022-01-03T01:00:10.399+01:00,1000_0005-CP01-2021-000063,EV,NaN,El Director General de Comunicación y Relacion...,"Departamento de Presidencia, Igualdad, Función...",Contrato del servicio de realización de labore...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 1379/2020 4738; Órgano de Contr...,Obres de renovació de l'enllumenat públic a la...,2022-01-03T00:11:40.740+01:00,1379/2020 4738,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Canet de Mar,Entitats municipals de Catalunya,Obres de renovació de l'enllumenat públic a la...,...,https://contractaciopublica.gencat.cat/ecofin_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 2021-44; Órgano de Contratación...,Subministre i la instal·lació fotovoltaica en ...,2022-01-03T00:11:40.696+01:00,2021-44,EV,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Valls,Entitats municipals de Catalunya,Subministre i la instal·lació fotovoltaica en ...,...,https://contractaciopublica.gencat.cat/ecofin_...,Enllac plec clausules tecniques.doc,https://contractaciopublica.gencat.cat/ecofin_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 2809/2021; Órgano de Contrataci...,Servei de visites i activitats sobre Història ...,2022-01-03T00:11:40.639+01:00,2809/2021,EV,https://contractaciopublica.gencat.cat/ecofin_...,Institut de Cultura de Barcelona,Entitats municipals de Catalunya,Servei de visites i activitats sobre Història ...,...,https://contractaciopublica.gencat.cat/ecofin_...,2809-21 PPT.pdf,https://contractaciopublica.gencat.cat/ecofin_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


...and, afterwards, a *hierarchical* version thereof

In [ ]:
hierarchical_df = flat_df_to_multiindexed_df(df)
hierarchical_df.head()

id  \
                                                       
                                                       
                                                       
                                                       
                                                       
0  https://contrataciondelestado.es/sindicacion/P...   
1  https://contrataciondelestado.es/sindicacion/P...   
2  https://contrataciondelestado.es/sindicacion/P...   
3  https://contrataciondelestado.es/sindicacion/P...   
4  https://contrataciondelestado.es/sindicacion/P...   

                                             summary  \
                                                       
                                                       
                                                       
                                                       
                                                       
0  Id licitación: C. 2-2021; Órgano de Contrataci...   
1  Id licitación: 8128_3/2021; Órgano de Contrata...   
2  Id licitación: 1000_0005-CP01-2021-000063; Órg...   
3  Id licitación: 1379/2020 4738; Órgano de Contr...   
4  Id licitación: 2021-44; Órgano de Contratación...   

                                               title  \
                                                       
                                                       
                                                       
                                                       
                                                       
0  L'objecte del contracte és la renovació de tot...   
1  Obras de restauración hidromorfológica del río...   
2  Contrato del servicio de realización de labore...   
3  Obres de renovació de l'enllumenat públic a la...   
4  Subministre i la instal·lació fotovoltaica en ...   

                         updated        ContractFolderStatus  \
                                            ContractFolderID   
                                                               
                                                               
                                                               
                                                               
0  2022-01-03T01:11:41.826+01:00                   C. 2-2021   
1  2022-01-03T01:00:11.194+01:00                 8128_3/2021   
2  2022-01-03T01:00:10.399+01:00  1000_0005-CP01-2021-000063   
3  2022-01-03T00:11:40.740+01:00              1379/2020 4738   
4  2022-01-03T00:11:40.696+01:00                     2021-44   

                                                                               \
  ContractFolderStatusCode                            LocatedContractingParty   
                                                            BuyerProfileURIID   
                                                                                
                                                                                
                                                                                
0                      ADJ  https://contractaciopublica.gencat.cat/ecofin_...   
1                      PUB                                                NaN   
2                       EV                                                NaN   
3                       EV  https://contractaciopublica.gencat.cat/ecofin_...   
4                       EV  https://contractaciopublica.gencat.cat/ecofin_...   

                                                      \
                                                       
                                               Party   
                                           PartyName   
                                                Name   
                                                       
0                           Ajuntament de Sant Ramon   
1                             Pleno del Ayuntamiento   
2  El Director General de Comunicación y Relacion...   
3                         Ajuntament de Canet de Mar   
4                                Ajuntamen

*Cross-sections* of the above `pd.DataFrame` can be extracted

In [ ]:
hierarchical_df.xs('id', axis='columns')

0      https://contrataciondelestado.es/sindicacion/P...
1      https://contrataciondelestado.es/sindicacion/P...
2      https://contrataciondelestado.es/sindicacion/P...
3      https://contrataciondelestado.es/sindicacion/P...
4      https://contrataciondelestado.es/sindicacion/P...
                             ...                        
112    https://contrataciondelestado.es/sindicacion/P...
113    https://contrataciondelestado.es/sindicacion/P...
114    https://contrataciondelestado.es/sindicacion/P...
115    https://contrataciondelestado.es/sindicacion/P...
116    https://contrataciondelestado.es/sindicacion/P...
Name: id, Length: 117, dtype: object

At deeper levels (notice in the above table *ContractFolderID* is nested inside *ContractFolderStatus*)

In [ ]:
hierarchical_df.xs('ContractFolderID', axis=1, level=1)

,ContractFolderStatus
,
,
,
,
0,C. 2-2021
1,8128_3/2021
2,1000_0005-CP01-2021-000063
3,1379/2020 4738
4,2021-44
...,...


The latter encompasses most of the data

In [ ]:
hierarchical_df.xs('ContractFolderStatus', axis=1)

ContractFolderID ContractFolderStatusCode  \
                                                           
                                                           
                                                           
                                                           
0                     C. 2-2021                      ADJ   
1                   8128_3/2021                      PUB   
2    1000_0005-CP01-2021-000063                       EV   
3                1379/2020 4738                       EV   
4                       2021-44                       EV   
..                          ...                      ...   
112               1005_391-2021                      PUB   
113                 8165_3/2021                       EV   
114                 8113_3/2021                       EV   
115                8113_01 2021                       EV   
116                0001264/2021                      RES   

                               LocatedContractingParty  \
                                     BuyerProfileURIID   
                                                         
                                                         
                                                         
0    https://contractaciopublica.gencat.cat/ecofin_...   
1                                                  NaN   
2                                                  NaN   
3    https://contractaciopublica.gencat.cat/ecofin_...   
4    https://contractaciopublica.gencat.cat/ecofin_...   
..                                                 ...   
112                                                NaN   
113                                                NaN   
114                                                NaN   
115                                                NaN   
116                                                NaN   

                                                        \
                                                 Party   
                                             PartyName   
                                                  Name   
                                                         
0                             Ajuntament de Sant Ramon   
1                               Pleno del Ayuntamiento   
2    El Director General de Comunicación y Relacion...   
3                           Ajuntament de Canet de Mar   
4                                  Ajuntament de Valls   
..                                                 ...   
112  Dirección General de Cultura-Institución Prínc...   
113  Mancomunidad de Servicios Sociales de Base de ...   
114                  Subdirector de Gestión y Recursos   
115  Agencia Navarra de Autonomía y Desarrollo de l...   
116  Agencia Pública Empresarial Sanitaria Bajo Gua...   

                                                        \
                                    ParentLocatedParty   
                                             PartyName   
                                                  Name   
                                                         
0                     Entitats municipals de Catalunya   
1                              AYUNTAMIENTO DE MONREAL   
2    Departamento de Presidencia, Igualdad, Función...   
3                     Entitats municipals de Catalunya   
4                     Entitats municipals de Catalunya   
..                                                 ...   
112        Departamento de Cultura, Deporte y Juventud   
113  MANCOMUNIDAD DE SERVICIOS DE HUARTE Y DE ESTER...   
114                Agencia Navarra para la Dependencia   
115                Agencia Navarra para la Dependencia   
116                                 Junta de Andalucía   

                                    ProcurementProject           \
                                                  Name TypeCode   
                                                                  
                                                                  

## Util

A function to produce a *full* index path from the upper levels. NOTE: it turns out that when loading  a `pd.DataFrame` saved in *parquet* format (using `to_parquet`), `np.nan` (a *float*) in a multiindexed column becomes `nan` (a *string*). This is accounted for in the code below.

In [ ]:
#| export
def pad_col_levels(
    df: pd.DataFrame, # Input
    levels: tuple | list, # Individual names to assemble and pad
    denan: bool = False # If `True`, skip `pd.NA`s
    ) -> tuple: # Multiindex column name
    "Builds a multiindex-amenable column name from a sequence of levels."
    
    # if "de-NaN" was requested...
    if denan:
        levels = [e for e in levels if pd.notna(e)]
    
    return tuple(list(levels) + [''] * (df.columns.nlevels - len(levels)))

A `tuple` representing column multiindex is built

In [ ]:
col_index = pad_col_levels(hierarchical_df, ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name'] )
col_index

('ContractFolderStatus',
 'LocatedContractingParty',
 'Party',
 'PartyName',
 'Name',
 '')

*nan*'s can be omitted by passing the corresponding parameter

In [ ]:
pad_col_levels(hierarchical_df, ['ContractFolderStatus', 'LocatedContractingParty', 'Party', 'PartyName', 'Name', np.nan, np.nan], denan=True)

('ContractFolderStatus',
 'LocatedContractingParty',
 'Party',
 'PartyName',
 'Name',
 '')

It is used to retrieve the corresponding column

In [ ]:
hierarchical_df.loc[:, col_index]

0                               Ajuntament de Sant Ramon
1                                 Pleno del Ayuntamiento
2      El Director General de Comunicación y Relacion...
3                             Ajuntament de Canet de Mar
4                                    Ajuntament de Valls
                             ...                        
112    Dirección General de Cultura-Institución Prínc...
113    Mancomunidad de Servicios Sociales de Base de ...
114                    Subdirector de Gestión y Recursos
115    Agencia Navarra de Autonomía y Desarrollo de l...
116    Agencia Pública Empresarial Sanitaria Bajo Gua...
Name: (ContractFolderStatus, LocatedContractingParty, Party, PartyName, Name, ), Length: 117, dtype: object

A function returning the columns containing a given `substring` in their (multi-level) name

In [ ]:
#| export
def columns_containing(df: pd.DataFrame, substring: str):
    
    is_contained = [list(filter(lambda e: (type(e) != float) and (substring in e), c)) != [] for c in df.columns]
    
    return df.columns[is_contained]

In [ ]:
columns_containing(hierarchical_df, 'Name')

MultiIndex([('ContractFolderStatus', 'LocatedContractingParty', ...),
            ('ContractFolderStatus', 'LocatedContractingParty', ...),
            ('ContractFolderStatus',      'ProcurementProject', ...),
            ('ContractFolderStatus',            'TenderResult', ...),
            ('ContractFolderStatus',         'ValidNoticeInfo', ...),
            ('ContractFolderStatus', 'LocatedContractingParty', ...)],
           )

A simple function to determine whether the columns of a `pd.DataFrame` are multiindexed.

In [ ]:
#| export
def is_column_multiindexed(
    df: pd.DataFrame # Input
    ) -> bool: # Assessment
    "Returns `True` if the given `pd.DataFrame` is column-multiindex."

    return type(df.columns) == pd.MultiIndex

In [ ]:
is_column_multiindexed(hierarchical_df)

True

In [ ]:
is_column_multiindexed(df)

False

### Flattening and renaming columns

The above `pd.DataFrame` with *multiindexed* columns

In [ ]:
hier_df = flat_df_to_multiindexed_df(df)
hier_df.head(2)

id  \
                                                       
                                                       
                                                       
                                                       
                                                       
0  https://contrataciondelestado.es/sindicacion/P...   
1  https://contrataciondelestado.es/sindicacion/P...   

                                             summary  \
                                                       
                                                       
                                                       
                                                       
                                                       
0  Id licitación: C. 2-2021; Órgano de Contrataci...   
1  Id licitación: 8128_3/2021; Órgano de Contrata...   

                                               title  \
                                                       
                                                       
                                                       
                                                       
                                                       
0  L'objecte del contracte és la renovació de tot...   
1  Obras de restauración hidromorfológica del río...   

                         updated ContractFolderStatus  \
                                     ContractFolderID   
                                                        
                                                        
                                                        
                                                        
0  2022-01-03T01:11:41.826+01:00            C. 2-2021   
1  2022-01-03T01:00:11.194+01:00          8128_3/2021   

                                                                               \
  ContractFolderStatusCode                            LocatedContractingParty   
                                                            BuyerProfileURIID   
                                                                                
                                                                                
                                                                                
0                      ADJ  https://contractaciopublica.gencat.cat/ecofin_...   
1                      PUB                                                NaN   

                                                               \
                                                                
                      Party                ParentLocatedParty   
                  PartyName                         PartyName   
                       Name                              Name   
                                                                
0  Ajuntament de Sant Ramon  Entitats municipals de Catalunya   
1    Pleno del Ayuntamiento           AYUNTAMIENTO DE MONREAL   

                                                      ...  \
                                  ProcurementProject  ...   
                                                Name  ...   
                                                      ...   
                                                      ...   
                                                      ...   
0  L'objecte del contracte és la renovació de tot...  ...   
1  Obras de restauración hidromorfológica del río...  ...   

                                                                       \
  LegalDocumentReference TechnicalDocumentReference                     
              Attachment                         ID        Attachment   
       ExternalReference                            ExternalReference   
                     URI                                          URI   
                                                                        
0                    NaN                        NaN               NaN   
1                    NaN                        NaN               NaN   

  

A mapping between columns and *human-readable* fields can be found [here](https://docs.google.com/spreadsheets/d/1vRN_bLWyUp6Hs1ONUSP9m3cfrgjx5bfbD4wzW4gXCu0/edit#gid=0). The latter was processed in `naming.ipynb` to get the file below.

In [ ]:
data_scheme_file = directory / 'PLACE.yaml'
assert data_scheme_file.exists()
data_scheme_file

PosixPath('/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/PLACE.yaml')

It provides (as a `dict`) a mapping from *human-readable* names to (maybe nested) fields in *Atom* files

In [ ]:
with open(data_scheme_file) as yaml_data:
    data_scheme = yaml.load(yaml_data, Loader=yaml.FullLoader)
{k: data_scheme[k] for k in list(data_scheme)[:5]}

{'id': ['id', nan, nan, nan, nan, nan, nan],
 'summary': ['summary', nan, nan, nan, nan, nan, nan],
 'title': ['title', nan, nan, nan, nan, nan, nan],
 'updated': ['updated', nan, nan, nan, nan, nan, nan],
 'Número de Expediente': ['ContractFolderStatus',
  'ContractFolderID',
  nan,
  nan,
  nan,
  nan,
  nan]}

A convenience function, `n2c` (as in "name to column"), to go from *human-readable* name to actual *multiindex* column

In [ ]:
n2c = lambda n: pad_col_levels(hier_df, data_scheme[n], denan=True)

The mapping can be exploited to access the columns of the `pd.DataFrame` using more natural names

In [ ]:
hier_df[n2c('Número de Expediente')]

0                       C. 2-2021
1                     8128_3/2021
2      1000_0005-CP01-2021-000063
3                  1379/2020 4738
4                         2021-44
                  ...            
112                 1005_391-2021
113                   8165_3/2021
114                   8113_3/2021
115                  8113_01 2021
116                  0001264/2021
Name: (ContractFolderStatus, ContractFolderID, , , , ), Length: 117, dtype: object

A hierarchical column *multiindex* is assumed.

In [ ]:
inv_data_scheme = {''.join([e if pd.notna(e) else '' for e in v]): k for k, v in data_scheme.items()}
[k for k in itertools.islice(inv_data_scheme, 4, 6)]

['ContractFolderStatusContractFolderID',
 'ContractFolderStatusContractFolderStatusCode']

In [ ]:
('id', '', '', '', '', '') in inv_data_scheme

False

In [ ]:
hier_df.columns.nlevels

6

In [ ]:
''.join(hier_df.columns[0])

'id'

A tiny function to ensure some data scheme is valid

In [ ]:
#| export
def _data_scheme_ok(data_scheme: dict) -> bool:
    
    lengths = []
    
    for l in data_scheme.values():
        
        lengths.append(len(l))
        
        # if not every element is a `str` or `nan`...
        if not np.all([(type(e) == str) or np.isnan(e) for e in l]):
            
            return False
        
    return True
    # return np.unique(lengths).shape[0] == 1 # <-------------------------- TODO: required?

In [ ]:
_data_scheme_ok(data_scheme)

True

In [ ]:
import copy

A data scheme with the wrong number of elements

In [ ]:
invalid_data_scheme = copy.deepcopy(data_scheme)
invalid_data_scheme['id'].append(np.nan)
len(invalid_data_scheme['id']), len(invalid_data_scheme['summary'])

(8, 7)

In [ ]:
_data_scheme_ok(inv_data_scheme)

True

A data scheme with the wrong type

In [ ]:
invalid_data_scheme = copy.deepcopy(data_scheme)
invalid_data_scheme['id'].append(2)

In [ ]:
_data_scheme_ok(inv_data_scheme)

True

A function to rename every *multi-indexed* column using the provided mapping (`data_scheme`).

In [ ]:
#| export
def flatten_columns_names(
    df: pd.DataFrame, # Input
    data_scheme: dict # Every key is a flattened name, and every value a list with the different levels of the multi-index
    , inplace: bool = False # If `True` the input DataFrame is modified
    ) -> None | pd.DataFrame: # Flat DataFrame
    
    assert _data_scheme_ok(data_scheme), f'data scheme is not OK'
    
    # the inverse of the above mapping (turning nan's into empty strings, and concatenating all the levels together)
    inv_data_scheme = {''.join([e if pd.notna(e) else '' for e in v]): k for k, v in data_scheme.items()}
    
    new_names = []
    
    for c in df.columns:
        
        stitched_c = ''.join(c)
        
        # if the column is found in the inverse mapping...
        if stitched_c in inv_data_scheme:
            
            # ...the given name is used
            new_names.append(inv_data_scheme[stitched_c])
            
        # if the columns is NOT found in the inverse mapping...
        else:
            
            # ...the new name is obtained by contatenating the individual components
            new_names.append(sproc.structure.nested_tags_separator.join([e for e in c if e != '']))
    
    if inplace:
        
       res = df
    
    else:
        
        res = df.copy()
    
    res.columns = new_names
    
    return res

In [ ]:
renamed_cols_df = flatten_columns_names(hier_df, data_scheme)
renamed_cols_df.head(2)

,id,summary,title,updated,Número de Expediente,Estado,URL perfil de contratante,Nombre,Ubicación orgánica,Objeto del Contrato,...,Pliego de cláusulas administrativas (URI),Pliego de Prescripciones técnicas,Pliego de Prescripciones técnicas (URI),Plazo de Ejecución (Comienzo),Plazo de Ejecución (Fin),ID,ContractFolderStatus - LocatedContractingParty - ParentLocatedParty - ParentLocatedParty - PartyName - Name,Presentación de Solicitudes (Fecha),Presentación de Solicitudes (Hora),Lote
0,https://contrataciondelestado.es/sindicacion/P...,Id licitación: C. 2-2021; Órgano de Contrataci...,L'objecte del contracte és la renovació de tot...,2022-01-03T01:11:41.826+01:00,C. 2-2021,ADJ,https://contractaciopublica.gencat.cat/ecofin_...,Ajuntament de Sant Ramon,Entitats municipals de Catalunya,L'objecte del contracte és la renovació de tot...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://contrataciondelestado.es/sindicacion/P...,Id licitación: 8128_3/2021; Órgano de Contrata...,Obras de restauración hidromorfológica del río...,2022-01-03T01:00:11.194+01:00,8128_3/2021,PUB,NaN,Pleno del Ayuntamiento,AYUNTAMIENTO DE MONREAL,Obras de restauración hidromorfológica del río...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


One can now used the *human-readable* columns names, e.g.

In [ ]:
renamed_cols_df['Número de Expediente']

0                       C. 2-2021
1                     8128_3/2021
2      1000_0005-CP01-2021-000063
3                  1379/2020 4738
4                         2021-44
                  ...            
112                 1005_391-2021
113                   8165_3/2021
114                   8113_3/2021
115                  8113_01 2021
116                  0001264/2021
Name: Número de Expediente, Length: 117, dtype: object

#### Insiders & Minors

The data scheme **for both**

In [ ]:
data_scheme_file = directory.parent / 'naming' / 'insiders_minors.yaml'
print(data_scheme_file)
assert data_scheme_file.exists()

/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/naming/insiders_minors.yaml


In [ ]:
with open(data_scheme_file) as yaml_data:
    data_scheme = yaml.load(yaml_data, Loader=yaml.FullLoader)
{k: data_scheme[k] for k in list(data_scheme)[:5]}

{'id': ['id'],
 'summary': ['summary'],
 'title': ['title'],
 'updated': ['updated'],
 'deleted_on': ['deleted_on']}

##### Insiders

In [ ]:
insiders_file = directory / 'insiders_sample.parquet'
print(insiders_file)
assert insiders_file.exists()

/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/insiders_sample.parquet


In [ ]:
insiders_df = pd.read_parquet(insiders_file)
insiders_df

id  \
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
zip                                               file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_2018.zip licitacionesPerfilesContratanteCompleto3_202005... 494    https://contrataciondelestado.es/sindicacion/l...   
                                                                                                     489    https://contrataciondelestado.es/sindicacion/l...   
                                                                                                     488    https://contrataciondelestado.es/sindicacion/l...   
                                                                                                     486    https://contrataciondelestado.es/sindicacion/l...   
                                                                                                     481    https://contrataciondelestado.es/sindicacion/l...   

                                                                                                                                                      summary  \
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                                                
                                                                                                                                 

In [ ]:
flatten_columns_names(insiders_df, data_scheme)

id  \
zip                                               file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_2018.zip licitacionesPerfilesContratanteCompleto3_202005... 494    https://contrataciondelestado.es/sindicacion/l...   
                                                                                                     489    https://contrataciondelestado.es/sindicacion/l...   
                                                                                                     488    https://contrataciondelestado.es/sindicacion/l...   
                                                                                                     486    https://contrataciondelestado.es/sindicacion/l...   
                                                                                                     481    https://contrataciondelestado.es/sindicacion/l...   

                                                                                                                                                      summary  \
zip                                               file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_2018.zip licitacionesPerfilesContratanteCompleto3_202005... 494    Id licitación: LICT/99/024/2017/0063; Órgano d...   
                                                                                                     489    Id licitación: 357/17; Órgano de Contratación:...   
                                                                                                     488    Id licitación: 7.1.23/17; Órgano de Contrataci...   
                                                                                                     486    Id licitación: 5/2016; Órgano de Contratación:...   
                                                                                                     481    Id licitación: 1004217003400; Órgano de Contra...   

                                                                                                                                                        title  \
zip                                               file name                                          entry                                                      
licitacionesPerfilesContratanteCompleto3_2018.zip licitacionesPerfilesContratanteCompleto3_202005... 494    Contratación de la Obra de creación de un nuev...   
                                                                                                     489          Suministro de mobiliario Hospital de Sagunt   
                                                                                                     488    Red separativa de alcantarillado en el barrio ...   
                                                                                                     486    Prestación del servicio de vigilancia y limpie...   
                                                                                                     481    Servicios de mantenimiento y desarrollo de los...   

                                                                                                                                    updated  \
zip                                               file name                                          entry                                    
licitacionesPerfilesContratanteCompleto3_2018.zip licitacionesPerfilesContratanteCompleto3_202005... 494   2017-12-29 12:36:06.402000+00:00   
                                                                                                     489   2017-12-29 12:44:12.934000+00:00   
                                                                                                     488   2017-12-29 12:45:04.741000+00:00   
                                                        

##### Minors

In [ ]:
minors_file = directory / 'minors_sample.parquet'
assert minors_file.exists()
print(minors_file)

/home/manu/Sync/UC3M/proyectos/2022/nextProcurement/sproc/samples/minors_sample.parquet


In [ ]:
minors_df = pd.read_parquet(minors_file)
minors_df

id  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
zip                                           file name                                          entry                                                      
contratosMenoresPerfilesContratantes_2018.zip contratosMenoresPerfilesContratantes_20190225_1... 499    https://contrataciondelestado.es/sindicacion/d...   
                                                                                                 498    https://contrataciondelestado.es/sindicacion/d...   
                                                                                                 497    https://contrataciondelestado.es/sindicacion/d...   
                                                                                                 496    https://contrataciondelestado.es/sindicacion/d...   
                                                                                                 495    https://contrataciondelestado.es/sindicacion/d...   

                                                                                                                                                  summary  \
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                                                                                                            
                                                                    

In [ ]:
flatten_columns_names(minors_df, data_scheme)

id  \
zip                                           file name                                          entry                                                      
contratosMenoresPerfilesContratantes_2018.zip contratosMenoresPerfilesContratantes_20190225_1... 499    https://contrataciondelestado.es/sindicacion/d...   
                                                                                                 498    https://contrataciondelestado.es/sindicacion/d...   
                                                                                                 497    https://contrataciondelestado.es/sindicacion/d...   
                                                                                                 496    https://contrataciondelestado.es/sindicacion/d...   
                                                                                                 495    https://contrataciondelestado.es/sindicacion/d...   

                                                                                                                                                  summary  \
zip                                           file name                                          entry                                                      
contratosMenoresPerfilesContratantes_2018.zip contratosMenoresPerfilesContratantes_20190225_1... 499    Id licitación: 000103/2017-1069; Órgano de Con...   
                                                                                                 498    Id licitación: 29-2017-II; Órgano de Contratac...   
                                                                                                 497    Id licitación: 013-07-2018; Órgano de Contrata...   
                                                                                                 496    Id licitación: CON/2017/51; Órgano de Contrata...   
                                                                                                 495    Id licitación: 000047/2017-1069; Órgano de Con...   

                                                                                                                                                    title  \
zip                                           file name                                          entry                                                      
contratosMenoresPerfilesContratantes_2018.zip contratosMenoresPerfilesContratantes_20190225_1... 499    Reforma de elementos de ventilación exterior d...   
                                                                                                 498    Servicios de calibrado y certificado de dos de...   
                                                                                                 497      Patrocinio menor proyecto " Activitats C.I.N.E"   
                                                                                                 496    Redacción EPIA - Legalización antena emisora d...   
                                                                                                 495    Obras de reparación del Centro de Información ...   

                                                                                                                                updated  \
zip                                           file name                                          entry                                    
contratosMenoresPerfilesContratantes_2018.zip contratosMenoresPerfilesContratantes_20190225_1... 499   2018-01-02 07:41:12.989000+00:00   
                                                                                                 498   2018-01-02 07:48:40.056000+00:00   
                                                                                                 497   2018-01-02 07:52:03.763000+00:00   
                                                                                                 496   2018-01-02 07:53:43.525000+00:00   
                 

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export

In [ ]:
#| hide
nbdev_export('30_hierarchical.ipynb')